<a href="https://colab.research.google.com/github/thejellis/nasaspaceapps2024clt_PACE_ppj/blob/instructionalDesign/curriculumchatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install fitz
# !pip install openai
# !pip install pypdf
# !pip install PyPDF2
#!pip install pdfplumber
#!pip install beautifulsoup4
#!pip install openai==0.28
#!pip install Pillow reportlab
#!pip install reportlab
#!pip install -Uq diffusers transformers fastcore

In [ ]:
import os
import openai
from bs4 import BeautifulSoup
from pypdf import PdfReader
import requests
import PyPDF2
import io
import requests

import pdfplumber

openai.api_key = os.getenv("OPENAIAPIKEY")

In [ ]:
def getinfo2(pdf_url, php_url):
    """
    Extracts text from a PDF URL and a website (PHP-generated content) URL and combines them.

    Args:
        pdf_url: The URL to the PDF file.
        php_url: The URL to the website containing PHP-generated content.

    Returns:
        A string containing the combined text from both sources.
    """

    # Step 1: Extract text from the PDF
    try:
        response_pdf = requests.get(pdf_url)
        pdf_file = 'temp.pdf'

        # Save the PDF locally
        with open(pdf_file, 'wb') as f:
            f.write(response_pdf.content)

        # Extract text from PDF
        all_pdf_text = ''
        with pdfplumber.open(pdf_file) as pdf:
            for page in pdf.pages:
                all_pdf_text += page.extract_text()

    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        all_pdf_text = "Error reading PDF content."

    # Step 2: Extract text from the PHP-generated website
    try:
        response_php = requests.get(php_url)
        soup = BeautifulSoup(response_php.content, 'html.parser')

        # Extract all visible text from the website
        all_php_text = soup.get_text(separator=' ', strip=True)

    except Exception as e:
        print(f"Error extracting text from PHP-generated website: {e}")
        all_php_text = "Error reading website content."

    # Combine both texts
    combined_text = all_pdf_text + "\n\n" + all_php_text
    return combined_text

# Example usage:
pdf_url = "https://pace.oceansciences.org/docs/earth_observer_jul_aug_2015.pdf"
php_url = "https://example.com/some_php_page"

combined_text = getinfo(pdf_url, php_url)
print(combined_text)


In [ ]:

def getinfo(mypdf):

# Step 1: Download the PDF from the URL
  pdf_url = mypdf
  response = requests.get(pdf_url)
  pdf_file = 'temp.pdf'

  # Save the PDF locally
  with open(pdf_file, 'wb') as f:
      f.write(response.content)

  # Step 2: Extract text using pdfplumber
  with pdfplumber.open(pdf_file) as pdf:
      all_text = ''
      for page in pdf.pages:
          all_text += page.extract_text()

  return all_text
  # Now `all_text` contains the text from the PDF

In [ ]:

def chatbot(mypdf, prompt):
  """
  Generates a response to a given prompt using OpenAI's ChatGPT API.

  Args:
      prompt: The user's prompt to the chatbot.

  Returns:
      The chatbot's response to the prompt.
  """
  pdfText = getinfo(mypdf)
  #pdfText = mypdf
  prompt = prompt + pdfText

  try:
        # Correct API method call
        completion =  openai.ChatCompletion.create( model="gpt-4",  # Change to the appropriate model if needed
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        return completion.choices[0].message['content'].strip()
  except Exception as e:
      print(f"Error communicating with OpenAI API: {e}")
      return "An error occurred. Please try again later."

#https://earthobservatory.nasa.gov/features/Aerosols
#https://pace.oceansciences.org/docs/ocean_color_brochure.pdf
#https://pace.oceansciences.org/docs/earth_observer_jul_aug_2015.pdf                               #
#https://earthobservatory.nasa.gov/features/Aerosols/page4.php
# https://earthobservatory.nasa.gov/features/Phytoplankton
# https://earthobservatory.nasa.gov/features/Aerosols
#https://earthobservatory.nasa.gov/features/Aerosols/page4.php
#https://earthobservatory.nasa.gov/features/Aerosols/page4.php

In [ ]:
prompt = "Create 3 weekly module curriculum with a data literacy optional module based on the following material: "
response = chatbot('https://pace.oceansciences.org/docs/NASA_TM2024219027v12_Level1C.pdf', prompt)
print(f"Chatbot: {response}")

In [ ]:
prompt1 = f"Create detailed lesson plans for the 3 weekly modules as a teacher guide from the following: {response} and : "
response1 = chatbot('https://pace.oceansciences.org/docs/NASA_TM2024219027v12_Level1C.pdf', prompt1)
print(f"Chatbot: {response1}")

**Image Generation with Prompts text**

Concept is to take the lesson plans and curriculum provided from the above generated information along with the NGSS, which provides the global science standards, will provide an opportunity for the teachers or other facilitators to generate custom content including books, brochures, or even presentations built eventually through quarto and rendered in html, books, or pdf

In [ ]:
import logging
from pathlib import Path

import matplotlib.pyplot as plt
import torch
from diffusers import StableDiffusionPipeline
from fastcore.all import concat
from huggingface_hub import notebook_login
from PIL import Image

logging.disable(logging.WARNING)

torch.manual_seed(1)
if not (Path.home()/'.cache/huggingface'/'token').exists(): notebook_login()

In [ ]:
print("Chatbot: Hello! What topic do you need to build materials for? : ")
prompts = []
  while True:
    topic = input("Chatbot: Hello! What topic do you need to build materials for? :")
    typeP = input("Which type do you need? (ie: presentation, document) : ")
    format = input("What is the intended audience (age and/or roles)? : ")
    length = input(" How long should it be?: ")
    location = input ("what language should the responses be?")
    otherinfo = input("What other information should be considered")
    submission = f("Create a {length} {typeP} for {format} about the {topic} in {location} considering {otherinfo}.")
    prompts.append()
    user_input = input('Do you want to submit another prompt? "exit", "quit", "bye"')
    if user_input.lower() in ["exit", "quit", "bye"]:
      print("Chatbot: Goodbye!")
      break
    for prompt in prompts:

      response = chatbot(prompt)
      print(f"Chatbot: {response}")

In [ ]:
prompt = "Create a short story for a picture book for 4th grader about the impact of algae blooms and why its important to be able to monitor and predict there."
response = chatbot('https://pace.oceansciences.org/docs/earth_observer_jul_aug_2015.pdf', prompt)
print(f"Chatbot: {response}")

In [ ]:
prompts = response.replace("?",".").replace("!",".").replace("\n","").replace('"',"").split(".")

newprompts = []
for prompt in prompts:
  prompt = prompt.strip()
  newprompts.append(prompt)

In [ ]:
"""
Once upon a time, in a beautiful world filled with sparkling waters and gentle breezes, there lived trillions of little friends called Phytoplankton. These tiny organisms danced gracefully in the deep blue oceans, rivers, and ponds all around our wonderful Planet Earth. They were like the lungs of the ocean, breathing life into our waters and providing us with half of the world's precious oxygen. Isn’t that just fantastic?

But, as in every story, change began to whisper in the winds. Phytoplankton started to grow a bit too much, creating what scientists like to call “algae blooms.” These blooms transformed the clear, tranquil waters into a thick, green soup, much like a vibrant painting gone wild. Sadly, this beautiful chaos began to suffocate the marine life, stealing away the oxygen they needed to thrive. Fishermen lost their cherished way of life as the fish vanished, and families became unwell from eating shellfish that had been touched by this change. Oh, dear, it was quite a serious challenge!

But don’t you worry, my friends, because up in the heavens, NASA scientists were hard at work. These clever folks, nestled in their labs far from the ocean’s embrace, had a brilliant solution brewing. They had been studying our planet’s shifting climate and were keeping a keen eye on those pesky algae blooms from the stars above using satellites. Imagine that!

This remarkable satellite, called PACE—short for Pre-Aerosols, Clouds, and ocean Ecosystems—had the amazing ability to see changes in the colors of the ocean. Just like a magic eye in the sky, it could tell where the blooms were sprouting up, helping the scientists understand where to look next. How wonderful is that?

But PACE didn’t stop there! It also watched how the ocean, clouds, and tiny particles in the air, called aerosols, danced together. This precious information helps scientists grasp how these algae blooms affect our weather, climate, and even the air we breathe. Just think of it as a beautiful symphony of nature, playing in harmony.

The findings and predictions from PACE were shared with our fishing friends, local leaders, and caring environmentalists. With this knowledge, they took steps to protect our marine life, safeguard human health, and even help preserve the livelihoods of our coastal communities. It’s a beautiful thing when we all come together for a common cause!

So, you see, watching and understanding these little algae blooms is just one wonderful example of how computer scientists, oceanographers, and the magic of space technology can join hands to help us live in harmony with our changing planet.

In the end, the story of algae blooms isn't just about tiny algae or the vast, majestic oceans. It’s about people, just like you and me, making thoughtful decisions and taking action for the health of our beautiful Earth. And with the help of tools like PACE, we can all be part of this lovely mission to keep our planet thriving."""

In [ ]:
count = 0
urls=[]
for prompt in newprompts:
  count +=1
  filename = f"storyimage_{str(count)}.jpg"
  url = generate_image(prompt)
  urls.append(url)
  urllib.request.urlretrieve(url, filename)




In [ ]:
prompts = ["A vibrant underwater scene showcasing trillions of tiny Phytoplankton glowing in clear blue waters,\n representing the lungs of the ocean."
, "A picturesque landscape with sparkling waters under a bright sun, depicting various bodies of water such as oceans, rivers, and ponds."
, "A stunning transformation scene showing clear waters turning into a thick, green algae bloom, resembling pea soup, with marine life struggling."
, "A fisherman looking concerned at his empty net on a fishing boat surrounded by algae blooms, illustrating the impact on livelihoods."
, "A worried family at the dinner table, glancing at shellfish, with a backdrop indicating food safety issues due to algae blooms."
,"A team of NASA scientists working in a modern lab filled with screens, studying Earth’s changing climate and algae blooms using satellite images."
,"An imaginative view of the PACE satellite in space, observing the color changes in the ocean from above."
, "A dramatic aerial view of the ocean with vibrant color differences, indicating algae blooms and healthy waters."
,"A beautiful landscape showing clouds and aerosols interacting with the ocean, depicting the complex relationship in the atmosphere."
,"A group of fishermen gathered around a digital display, learning about algae bloom predictions from scientists."
,"A marine biologist gently rescuing a fish from polluted waters, showcasing efforts to protect marine life affected by algae blooms."
,"A community meeting at a coastal town discussing the impact of algae blooms, with concerned citizens and local leaders."
, "A whimsical illustration of nature as a symphony, with elements of the ocean, clouds, and life coming together harmoniously."
, "Curious children learning about algae blooms in a classroom, with charts and pictures of the ocean displayed."
,"A close-up view of Phytoplankton under a microscope, showcasing their intricate beauty and structure."
, "A split image showing a healthy, vibrant ocean on one side and a polluted ocean with algae blooms on the other."
, "A research team on a boat collecting water samples to study algae blooms, surrounded by beautiful ocean scenery."
,"A colorful underwater scene depicting a variety of marine life thriving in clear waters, highlighting the importance of healthy ecosystems."
,"An awe-inspiring view of Earth from space, focusing on the oceans and the delicate balance of life."
,"A scientist exploring climate change impacts in a lab, with maps and data visualizations displayed."
, "A digital representation of air quality monitoring, showing the effects of aerosols and algae on human health."
,"An interactive learning exhibit about algae blooms at a science museum, with children engaging with displays."
,"A group of conservationists planting coral reefs, emphasizing the importance of restoring marine ecosystems."
, "A visual representation of how algae blooms affect weather patterns, with clouds and ocean currents illustrated."
, "A vibrant, healthy ocean ecosystem, teeming with life, showcasing the balance of marine and terrestrial environments."
, "A community gathering where people are pledging to take action for a healthier planet, emphasizing collective responsibility."
, "A futuristic depiction of technology being used in marine research, such as drones and underwater robots."
, "Children exploring a coastal area, learning about marine life and the effects of pollution on their environment."
, "A beautifully illustrated marine ecosystem showing the interdependence of various species, from phytoplankton to larger fish."
, "A globe with various regions highlighted, showing the global impact of climate change on oceans and marine life."
, "A hopeful scene of people taking action, such as beach clean-ups and conservation efforts, promoting positive change."
, "A serene sunset over the ocean, symbolizing hope and a reflection on the importance of caring for our planet."]

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", variant="fp16", torch_dtype=torch.float16).to("cuda")


In [ ]:
# Create PDF
# Function to add a page with text and image
def add_story_page(c, image_path, text, x, y, width, height):
    # Insert image
    c.drawImage(image_path, x, y, width, height)

    # Add text below or above the image
    text_y = y - 40 if y > height else y + height + 20
    c.setFont("Helvetica", 12)
    c.drawString(x, text_y, text)

# Create PDF
def create_storybook(images, texts, output_filename):
    c = canvas.Canvas(output_filename, pagesize=letter)
    width, height = letter

    # Loop through each image and text to create pages
    for image_path, text in zip(images, texts):
        img = Image.open(image_path)
        img_width, img_height = img.size

        # Scale the image to fit within the page
        aspect_ratio = img_height / img_width
        img_display_width = width - 100
        img_display_height = img_display_width * aspect_ratio

        # Add page content
        add_story_page(c, image_path, text, 50, height - img_display_height - 150, img_display_width, img_display_height)

        # Add a new page after each image and text
        c.showPage()

    # Save PDF
    c.save()


# Sample usage
images = imagejpgs
texts = prompts  # Corresponding text for each image
create_storybook(images, texts, "FinalStoryshort.pdf")



In [ ]:
images = []
imagejpgs = []
count=0
for prompt in prompts:
  count+=1
  torch.manual_seed(1024)
  image = pipe(prompt, num_inference_steps=16).images[0]
  images.append(image)
  path = f'image_{str(count)}.jpg'
  image.save(f'image_{str(count)}.jpg')
  imagejpgs.append(path)
